# 📏 Age Group Benchmarking — Percentile Rankings

## Key Findings
- Scoring increases dramatically with age: **U10 median ~2 PPG** vs **U18 median ~8+ PPG**
- The **spread widens** at older age groups — talent differentiation increases
- **90th percentile** at each age group represents true standout performance
- These benchmarks answer the question: *"Is my kid's scoring actually good for their age?"*

---

In [ ]:
%matplotlib inline
import sqlite3, re, pandas as pd, numpy as np
import matplotlib.pyplot as plt, seaborn as sns
import warnings; warnings.filterwarnings('ignore')

sns.set_theme(style="whitegrid"); plt.rcParams['figure.dpi'] = 120

conn = sqlite3.connect("../data/playhq.db")
df = pd.read_sql("""
    SELECT p.id as pid, p.first_name || ' ' || p.last_name as name,
        ps.games_played as gp, ps.total_points as pts, ps.total_fouls as fouls,
        g.name as grade
    FROM player_stats ps
    JOIN players p ON p.id = ps.player_id
    JOIN grades g ON g.id = ps.grade_id
    WHERE ps.games_played > 0
""", conn)
conn.close()

df["ppg"] = df["pts"] / df["gp"]
df["age_group"] = df["grade"].str.extract(r'(U\d+)')
df["gender"] = df["grade"].apply(lambda x: "Boys" if x.startswith("Boys") else ("Girls" if x.startswith("Girls") else "Unknown"))

# Aggregate per player per age group
agg = df.groupby(["pid", "name", "age_group", "gender"]).agg(
    gp=("gp", "sum"), pts=("pts", "sum"), fouls=("fouls", "sum")
).reset_index()
agg["ppg"] = agg["pts"] / agg["gp"]
agg["fpg"] = agg["fouls"] / agg["gp"]
agg = agg[agg["gp"] >= 3]

print(f"Player-age-groups (3+ games): {len(agg):,}")

## Percentile Benchmarks by Age Group

In [ ]:
pcts = [10, 25, 50, 75, 90, 95, 99]
rows = []
for (ag, gen), g in agg.groupby(["age_group", "gender"]):
    if ag is None or len(g) < 20:
        continue
    row = {"Age Group": ag, "Gender": gen, "N": len(g)}
    for p in pcts:
        row[f"P{p}"] = round(np.percentile(g["ppg"], p), 2)
    rows.append(row)

bench = pd.DataFrame(rows)
bench = bench.sort_values(["Gender", "Age Group"])
bench

In [ ]:
boys = agg[(agg["gender"] == "Boys") & agg["age_group"].notna()]
age_order = sorted(boys["age_group"].unique(), key=lambda x: int(x[1:]))

fig, axes = plt.subplots(1, 2, figsize=(16, 7))
fig.suptitle("Age Group Benchmarks — PPG Distributions (Boys)", fontsize=15, fontweight="bold")

# Box plot
ax = axes[0]
box_data = [boys[boys["age_group"] == ag]["ppg"].values for ag in age_order]
bp = ax.boxplot(box_data, labels=age_order, patch_artist=True, showfliers=False)
cmap = plt.cm.viridis(np.linspace(0.2, 0.9, len(age_order)))
for patch, color in zip(bp["boxes"], cmap):
    patch.set_facecolor(color)
ax.set_title("PPG by Age Group (3+ games)"); ax.set_xlabel("Age Group"); ax.set_ylabel("PPG")

# Percentile trend lines
ax = axes[1]
bb = bench[bench["Gender"] == "Boys"].copy()
bb["age_num"] = bb["Age Group"].str.extract(r"U(\d+)").astype(int)
bb = bb.sort_values("age_num")
ax.plot(bb["Age Group"], bb["P50"], "o-", color="#1976D2", lw=2, label="50th (median)")
ax.plot(bb["Age Group"], bb["P75"], "s--", color="#FF9800", lw=1.5, label="75th")
ax.plot(bb["Age Group"], bb["P90"], "^--", color="#D32F2F", lw=1.5, label="90th")
ax.plot(bb["Age Group"], bb["P95"], "d--", color="#7B1FA2", lw=1.5, label="95th")
ax.fill_between(bb["Age Group"], bb["P25"], bb["P75"], alpha=0.15, color="#1976D2")
ax.set_title("PPG Percentiles by Age Group"); ax.set_xlabel("Age Group"); ax.set_ylabel("PPG")
ax.legend(); ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig("../assets/age_benchmarks.png", dpi=150, bbox_inches="tight")
plt.show()

## How to Read These Benchmarks

| Percentile | Meaning |
|:---:|:---|
| **50th** | Median — half of players score more, half score less |
| **75th** | Above average — top quarter of the age group |
| **90th** | Standout — only 10% score higher |
| **95th** | Elite — top 5% of the age group |
| **99th** | Exceptional — one-in-a-hundred player |